In [35]:
from torchvision.ops import box_iou
import os
import pandas as pd
import json
import torch

In [36]:
BASE_DIR = "/mnt/Enterprise/safal/AI_assisted_microscopy_system/"
sample_types = [
    "smartphone_sample",
    # "smartphone_reference",
    # "brightfield_sample",
    # "brightfield_reference",
]
model_type = "yolov8"
sample_type="brightfield_sample"

In [37]:
gt_annotations_file = os.path.join(
    BASE_DIR,
    "cysts_dataset_all",
    sample_type,
    "fold_5",
    f"{sample_type}_coco_annos_val.json",
)
pred_annotations_file = os.path.join(
    BASE_DIR,
    "outputs",
    sample_type,
    model_type,
    "fold_5",
    "mmdetection_cysts",
    f"yolov8_{sample_type}_fold_5",
    "predictions.json"
)


In [38]:
gt_annos = json.load(open(gt_annotations_file))

In [39]:
pred_annos_df = pd.read_json(pred_annotations_file)

In [40]:
# change bbox format to x1, y1, x2, y2
pred_annos_df["bbox"] = pred_annos_df["bbox"].apply(
    lambda x: [x[0], x[1], x[0] + x[2], x[1] + x[3]]
)

In [41]:
# change gt_annos id value to image names
for i in range(len(gt_annos["images"])):
    gt_annos["images"][i]["image_id"] = gt_annos["images"][i]["file_name"].rsplit("/")[-1].split(".")[0]


In [42]:
images_df = pd.DataFrame(gt_annos["images"])
gt_annos_df = pd.DataFrame(gt_annos["annotations"])

In [43]:
images_df

,id,file_name,height,width,image_id
0,1,/mnt/Enterprise/safal/AI_assisted_microscopy_s...,480,640,BS0001
1,4,/mnt/Enterprise/safal/AI_assisted_microscopy_s...,480,640,BS0004
2,13,/mnt/Enterprise/safal/AI_assisted_microscopy_s...,480,640,BS0013
3,14,/mnt/Enterprise/safal/AI_assisted_microscopy_s...,480,640,BS0014
4,20,/mnt/Enterprise/safal/AI_assisted_microscopy_s...,480,640,BS0020
...,...,...,...,...,...
196,1040,/mnt/Enterprise/safal/AI_assisted_microscopy_s...,480,640,BS0983
197,1044,/mnt/Enterprise/safal/AI_assisted_microscopy_s...,480,640,BS0987
198,1046,/mnt/Enterprise/safal/AI_assisted_microscopy_s...,480,640,BS0989
199,1056,/mnt/Enterprise/safal/AI_assisted_microscopy_s...,480,640,BS0999


In [44]:
gt_annos_df["bbox"] = gt_annos_df["bbox"].apply(
    lambda x: [x[0], x[1], x[0] + x[2], x[1] + x[3]]
)


In [45]:
gt_annos_df

,image_id,id,category_id,bbox,segmentation,iscrowd,area
0,1,2,0,"[268.0, 432.0, 278.0, 442.0]",[],0,100.000000
1,1,3,0,"[518.0, 361.0, 530.0, 371.0]",[],0,120.000000
2,4,5,1,"[529.2140833378212, 297.5634355704484, 562.785...",[],0,835.036536
3,4,6,1,"[466.9134134481312, 374.45361413506225, 487.08...",[],0,586.893520
4,13,30,1,"[361.11871973472756, 168.8558744429978, 398.88...",[],0,917.186553
...,...,...,...,...,...,...,...
216,963,1012,1,"[241.91607846833455, 219.5085686776027, 266.08...",[],0,748.788962
217,1014,1061,0,"[319.0, 303.0, 331.0, 317.0]",[],0,168.000000
218,1016,1063,0,"[383.0, 165.0, 401.0, 183.0]",[],0,324.000000
219,1046,1080,0,"[498.0, 332.0, 512.0, 346.0]",[],0,196.000000


In [46]:
# replace image_id of gt_annos_df with image_id of images_df
gt_annos_df["image_id"] = gt_annos_df["image_id"].apply(
    lambda x: images_df[images_df["id"] == x]["image_id"].values[0]
)


In [47]:
gt_annos_df

,image_id,id,category_id,bbox,segmentation,iscrowd,area
0,BS0001,2,0,"[268.0, 432.0, 278.0, 442.0]",[],0,100.000000
1,BS0001,3,0,"[518.0, 361.0, 530.0, 371.0]",[],0,120.000000
2,BS0004,5,1,"[529.2140833378212, 297.5634355704484, 562.785...",[],0,835.036536
3,BS0004,6,1,"[466.9134134481312, 374.45361413506225, 487.08...",[],0,586.893520
4,BS0013,30,1,"[361.11871973472756, 168.8558744429978, 398.88...",[],0,917.186553
...,...,...,...,...,...,...,...
216,BS0906,1012,1,"[241.91607846833455, 219.5085686776027, 266.08...",[],0,748.788962
217,BS0957,1061,0,"[319.0, 303.0, 331.0, 317.0]",[],0,168.000000
218,BS0959,1063,0,"[383.0, 165.0, 401.0, 183.0]",[],0,324.000000
219,BS0989,1080,0,"[498.0, 332.0, 512.0, 346.0]",[],0,196.000000


In [48]:
pred_annos_df

,image_id,category_id,bbox,score
0,BS0001,0,"[518.5, 357.75, 530.5, 369.75]",0.42822
1,BS0001,0,"[267.75, 430.25, 278.75, 441.75]",0.21667
2,BS0001,0,"[518.0, 357.5, 528.0, 367.5]",0.07288
3,BS0001,0,"[535.5, 349.0, 546.5, 358.5]",0.00800
4,BS0001,0,"[517.0, 357.5, 531.0, 372.5]",0.00661
...,...,...,...,...
1976,BS0313,0,"[116.125, 353.0, 126.375, 364.0]",0.00220
1977,BS0313,0,"[320.5, 117.25, 331.0, 127.25]",0.00121
1978,BS0313,0,"[117.125, 354.5, 125.875, 362.5]",0.00110
1979,BS1003,0,"[284.5, 285.0, 299.5, 302.0]",0.57861


In [49]:
categories = sorted(gt_annos_df.category_id.unique())

    # dataframe to store the precision, recall and f1 score for each class
metrics_df = pd.DataFrame(
    columns=["category", "precision", "recall", "f1_score", "TP", "FP"]
)

In [50]:
for category in categories:
    # get the ground truth annotations for the current class
    gt_annos_df_class = gt_annos_df[
        gt_annos_df.category_id == category
    ]
    # get the predicted annotations for the current class
    pred_annos_df_class = pred_annos_df[
        pred_annos_df.category_id == category
    ]

    # sort the predicted annotations by score
    pred_annos_df_class = pred_annos_df_class.sort_values(
        by="score", ascending=False
    )

    # filter predictions with score > 0.3
    pred_annos_df_class = pred_annos_df_class[
        pred_annos_df_class.score > 0.1
    ]

    true_positives_class = 0
    false_positives_class = 0

    # get image ids for the current class from both ground truth and predicted annotations
    image_ids = pred_annos_df_class["image_id"].unique()
    images_len = len(image_ids)

    for image in image_ids:
        # get the ground truth annotations for the current image
        gt_annos_df_image = gt_annos_df_class[
            gt_annos_df_class.image_id == image
        ]
        # get the predicted annotations for the current image
        pred_annos_df_image = pred_annos_df_class[
            pred_annos_df_class.image_id == image
        ]

        # get the ground truth bounding boxes
        gt_bboxes = list(gt_annos_df_image.bbox.values)
        gt_bboxes = torch.tensor(gt_bboxes)

        # get the predicted bounding boxes
        pred_bboxes = list(pred_annos_df_image.bbox.values)
        pred_bboxes = torch.tensor(pred_bboxes)

        if len(gt_bboxes) == 0:
            false_positives_class += len(pred_bboxes)
            continue

        # get the intersection over union for each predicted bounding box
        ious = box_iou(pred_bboxes, gt_bboxes)

        # get the maximum iou for each ground truth bounding box
        max_ious, _ = torch.max(ious, dim=0)

        # get the indices of the predicted bounding boxes with iou > 0.5
        tp_indices = torch.where(max_ious >= 0.5)[0]
        # print(ious)

        # get the indices of the predicted bounding boxes with iou < 0.5
        fp_indices = torch.where(max_ious < 0.5)[0]

        # update the true positives and false positives
        true_positives_class += len(tp_indices)
        false_positives_class += len(fp_indices)

    # calculate the precision and recall
    precision = true_positives_class / (
        true_positives_class + false_positives_class
    )
    recall = true_positives_class / gt_annos_df_class.shape[0]

    category_name = gt_annos["categories"][category]["name"]
    f1_score = 2 * (precision * recall) / (precision + recall)

    category_metrics_df = pd.DataFrame(
        {
            "category": category_name,
            "precision": precision,
            "recall": recall,
            "f1_score": f1_score,
            "TP": true_positives_class,
            "FP": false_positives_class,
        },
        index=[0],
    )

    # concatenate the metrics for the current class to the metrics dataframe
    metrics_df = pd.concat([metrics_df, category_metrics_df], axis=0)


In [51]:
metrics_df

,category,precision,recall,f1_score,TP,FP
0,Crypto,0.803571,0.882353,0.841121,135,33
0,Giardia,0.762500,0.897059,0.824324,61,19
